In [1]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import gc
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import time
import seaborn as sns
import torchmetrics
from torchvision import transforms
from torchsummary import summary
import gc
from tqdm import tqdm
import cv2 as cv
import torch
from torch import nn


In [2]:
df = pd.read_csv("data/Data_Entry_2017.csv")
# df = df.drop(df[df["Finding Labels"]=="No Finding"].index)

In [3]:
def disp_nan(df): 
    nb_nan = pd.DataFrame([df[c].isnull().value_counts() for c in df.columns], index = df.columns)
    nb_nan.rename(columns={"False" : "Not NaN", "True" : "NaN"})
    return nb_nan

disp_nan(df)

,False
Image Index,112120
Finding Labels,112120
Follow-up #,112120
Patient ID,112120
Patient Age,112120
Patient Gender,112120
View Position,112120
OriginalImage[Width,112120
Height],112120
OriginalImagePixelSpacing[x,112120


In [4]:
df = df.dropna()

In [5]:
# display all chest diseases
# print(df['Finding Labels'].value_counts())

diagnostic = df["Finding Labels"]
labels = np.unique(diagnostic)
all_labels = []
for x in labels : 
    all_labels += x.split('|')
labels = np.unique(all_labels)
s = ''
for i,l in enumerate(labels) : 
    s+=f'\n {i} - '+l
print(f"{len(labels)} unique chest diseases : "+s)



15 unique chest diseases : 
 0 - Atelectasis
 1 - Cardiomegaly
 2 - Consolidation
 3 - Edema
 4 - Effusion
 5 - Emphysema
 6 - Fibrosis
 7 - Hernia
 8 - Infiltration
 9 - Mass
 10 - No Finding
 11 - Nodule
 12 - Pleural_Thickening
 13 - Pneumonia
 14 - Pneumothorax


In [6]:
df_finding_labels = df["Finding Labels"].str.get_dummies('|')
df = pd.concat([df,df_finding_labels], axis = 1)
df = df.drop('Finding Labels', axis = 1)

In [7]:
df_useful = pd.concat([df[['Image Index']], df_finding_labels], axis = 1)
file_path = 'my_data/data.csv'


# Save the DataFrame to CSV
df_useful.to_csv(file_path)
df = df_useful

In [8]:
df = pd.read_csv('my_data/data.csv', index_col='Unnamed: 0')
df


,Image Index,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
112116,00030802_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112117,00030803_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112118,00030804_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [9]:
df_finding_labels = df.iloc[:,1:]
df_finding_labels

,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
112116,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112117,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112118,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [10]:
# get images
import glob
images_names = np.array(glob.glob('data/images_*/images/*.png'))[df.index]
N = len(images_names)
print("The dataset contains {} images.".format(N))
images_names


The dataset contains 112120 images.


array(['data\\images_001\\images\\00000001_000.png',
       'data\\images_001\\images\\00000001_001.png',
       'data\\images_001\\images\\00000001_002.png', ...,
       'data\\images_012\\images\\00030803_000.png',
       'data\\images_012\\images\\00030804_000.png',
       'data\\images_012\\images\\00030805_000.png'], dtype='<U39')

In [11]:
images_names_train, images_names_test, labels_train, labels_test = train_test_split(images_names, df_finding_labels , test_size = 0.05)

In [12]:
# IMG_SIZE = 256


# class NormalizeImage(object):
#     def __call__(self, img):
#         mean = torch.mean(img)
#         std = torch.std(img)
#         return (img - mean) / std

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize(IMG_SIZE), 
#     NormalizeImage()])

In [14]:
IMG_SIZE = 256

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE)])


images_train = []
with tqdm(images_names_train, unit='images', total = len(images_names_train)) as tload :
        for x in tload : 
                im  = transform(cv.imread(x)[:,:,0])
                images_train.append(im)
                # time.sleep(1)

images_train = torch.stack(images_train)


 46%|████▋     | 49454/106514 [27:24<36:15, 26.23images/s]   

In [ ]:
images_train.size()

In [ ]:
mean = images_train.mean()
std = images_train.std()
print(mean, std)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std = std)])

images_train_norm = []
with tqdm(images_train, unit='images', total = len(images_train)) as tload :
        for x in tload : 
                im  = transform(x)
                images_train_norm.append(im)

images_train_norm = torch.stack(images_train_norm)



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transform.Resize(IMG_SIZE),
    transform.Normalize(mean = mean, std = std)])

images_test_norm = []

with tqdm(images_names_test, unit='images', total = len(images_names_test)) as tload :
        for x in tload : 
                im  = transform(cv.imread(x[:,:,0]))
                images_test_norm.append(im)

images_test_norm = torch.stack(images_test_norm)
